# Part 1 - Gathering YouTube Channel Data

In [ ]:
#import libraries
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import os

In [ ]:
#Gathers views, likes and comments for a particular video ID passes by get_videos
#Returns these metrics

def video_detail(video_id):
        video_stats_url = f'https://www.googleapis.com/youtube/v3/videos?key={API_KEY}&part=statistics&id='+video_id
        response_video_stats = requests.get(video_stats_url).json()
        views = response_video_stats['items'][0]['statistics']['viewCount']
        likes = response_video_stats['items'][0]['statistics']['likeCount']
        comments = response_video_stats['items'][0]['statistics']['commentCount']

        return views, likes, comments

In [ ]:
#Retrieves the description, caption and definition data by calling on the contentDetails property of API key
#for a particular video
#Returns these metrics

def get_misc_detail(video_id):
        misc_info_url = f'https://www.googleapis.com/youtube/v3/videos?key={API_KEY}&part=contentDetails&id='+video_id
        response_misc_info = requests.get(misc_info_url).json()

        description = response_misc_info['items'][0]['contentDetails']['duration']
        caption = response_misc_info['items'][0]['contentDetails']['caption']
        definition = response_misc_info['items'][0]['contentDetails']['definition']

        return description, caption, definition

In [ ]:
#Retriveves all videos from a Youtube channel

def get_videos(df, url):
    response = requests.get(url).json()

    for video in response['items']:
        if video['id']['kind'] == "youtube#video":
            video_id = video['id']['videoId']
            video_title = video['snippet']['title']
            video_title = str(video_title).replace("&amp;","and")
            upload_date = video['snippet']['publishTime']
            upload_date = str(upload_date).split("T")[0]

            views, likes, comments = video_detail(video_id)
            description, caption, definition = get_misc_detail(video_id)

            df = df.append({'video_id':video_id, 'video_title':video_title, 'upload_date': upload_date,
                            'views': views,'likes': likes,'comments': comments, 'description': description,
                            'caption':caption, 'definition':definition}, ignore_index=True)

    #If a new set of video results is present through the nextPageToken
    #property, updates the URL and calls function on itself  
    if "nextPageToken" in response:
        pageToken = response['nextPageToken']
        url = f'https://www.googleapis.com/youtube/v3/search?key={API_KEY}&channelId={CHANNEL_ID}&part=snippet,id&order=date&maxResults=50&pageToken='+pageToken
        #url = f'https://www.googleapis.com/youtube/v3/search?key={API_KEY}&channelId={CHANNEL_ID}&part=snippet,id&order=date&maxResults=50&pageToken='+pageToken
        df = get_videos(df, url)   
    return df

In [ ]:
#Keys
API_KEY = os.getenv("API_KEY")

CHANNEL_ID = os.getenv("CHANNEL_ID")

In [ ]:
#Creating a dataframe from data columns pulled from API calls
df = pd.DataFrame(columns=['video_id', 'video_title','upload_date','views','likes','comments','description','caption','definition'])

#Inital set of video results URL
pageToken = ""
url = f'https://www.googleapis.com/youtube/v3/search?key={API_KEY}&channelId={CHANNEL_ID}&part=snippet,id&order=date&maxResults=50&pageToken='+pageToken

df = get_videos(df,url)

# Part 2 - Cleaning Data

In [ ]:
df.head()

In [ ]:
#Checking to see if data has successfully been pulled
df.shape

In [ ]:
#Checking column datatypes for DF
df.dtypes

In [ ]:
#Cleaning DF data types
df['views'] = df.views.astype('int64')
df['likes'] = df.likes.astype('int64')
df['comments'] = df.comments.astype('int64')

df['upload_date'] = pd.to_datetime(df['upload_date'])
df['year'] = pd.DatetimeIndex(df['upload_date']).year
df['month'] = pd.DatetimeIndex(df['upload_date']).month

df.rename(columns={'description': 'duration'}, inplace=True)

df.dtypes


In [ ]:
df.head(5)

In [ ]:
import re

def YTDurationToSeconds(duration):
    match = re.match('PT((\d+)H)?((\d+)M)?((\d+)S)?', duration).groups()
    hours = int(match[1]) if match[1] else 0
    minutes = int(match[3]) if match[3] else 0
    seconds = int(match[5]) if match[5] else 0
    return hours * 3600 + minutes * 60 + seconds

In [ ]:
#Converting 'duration' into total seconds as INT
df['duration_seconds'] = df.iloc[0:,6].apply(YTDurationToSeconds)
df['duration_seconds'].dtype
    

In [ ]:
#Ensuring no duplicate videos, drop if needed
df.value_counts(['video_id'])

In [ ]:
df.head(5)

Now that we've clean our data for analysis, we can create a database to store a YouTube channel's video info!


# Part 3 - Storing Data to Database for Future Use
## Use Amazon AWS RDB (Postgres) for free database storage!

In [ ]:
#Installing packages
%pip install psycopg2
import psycopg2 as ps

In [ ]:
#Connecting to AWS Database

def connect_database(hostname, user, password, port, dbname):
    try:
        connection = ps.connect(host=hostname,user=user,password=password,port=port,database=dbname)
    except ps.OperationalError as e:
        raise e
    else:
        print('Successfully Connected to AWS Database :)')
    return connection

In [ ]:
#Creating SQL table statement
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    views INTEGER NOT NULL,
                    likes INTEGER NOT NULL,
                    comments INTEGER NOT NULL,
                    duration TEXT NOT NULL,
                    caption TEXT NOT NULL,
                    definition TEXT NOT NULL,
                    year INTEGER NOT NULL,
                    month INTEGER NOT NUll,
                    duration_seconds INTEGER NOT NULL
                )""")
    curr.execute(create_table_command)

In [ ]:
def insert_into_table(curr, video_id, video_title, upload_date, views, likes, comments, duration,
                      caption, definition, year, month, duration_seconds):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                        views, likes, comments, duration, caption, definition, year,
                        month, duration_seconds)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, views, likes, comments, duration, caption, definition, year,
                        month, duration_seconds)
    curr.execute(insert_into_videos, row_to_insert)

In [ ]:
#update row if video exists
def update_row(curr, video_id, video_title, views, likes, comments, duration, caption, definition, \
               year, month, duration_seconds):
    query = ("""UPDATE videos
                SET video_title = %s,
                    views = %s,
                    like = %s,
                    comments = %s,
                WHERE video_id = %s;""")

    update_vars = (video_title, views, likes, comments, video_id, duration, caption, definition,
                   year, month, duration_seconds)
    curr.execute(query, update_vars)

In [ ]:
def video_exists(curr, video_id):
    query = ("""SELECT video_id FROM videos WHERE video_id = %s""")
    curr.execute(query, (video_id,))

    return curr.fetchone() is not None

In [ ]:
def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")

    curr.execute(truncate_table)

In [ ]:
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['views']
                            , row['likes'], row['comments'], row['duration'], row['caption'], row['definition'],
                                row['year'], row['month'], row['duration_seconds'])

In [ ]:
def update_db(curr, df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'views', 'likes', 'comments',
                                'duration', 'caption', 'definition', 'year', 'month', 'duration_seconds'])

    for i, row in df.iterrows():
        if video_exists(curr, row['video_id']):
            update_row(curr, row['video_id'], row['video_title'], row['views'], row['likes'], row['comments']
                        , row['duration'], row['caption'], row['definition'],
                            row['year'], row['month'], row['duration_seconds'])
        else:
            tmp_df = tmp_df.append(row)

    return tmp_df

In [ ]:
#Getting DB credentials
hostname = os.getenv("hostname")
user = os.getenv("user")
password = os.getenv("password")
port = '5432'
dbname = os.getenv("dbname")
connection = None

connection = connect_database(hostname, user, password, port, dbname)
curr = connection.cursor()

In [ ]:
create_table(curr)

In [ ]:
new_df = update_db(curr, df)
connection.commit()

In [ ]:
append_from_df_to_db(curr, new_df)
connection.commit()

In [ ]:
curr.execute("SELECT * FROM videos")